# Throughputs_LSSTobs_CalSpec

Based on Lynne Jones code here: https://rubin-obs.slack.com/archives/C0824CTA335/p1732311332938929

Authors:  C. L. Adair, D. L. Tucker, with help from L. Jones, J. Carlin, E. Rykoff, R. Lupton, and others

Created:  2024.11.27

Updated:  2025.10.20

## 1. Initial Setup...

### 1.1 Import useful python packages

In [ ]:
# Generic python packages
import pylab as plt
import numpy as np
import pandas as pd
import glob
import math
import os
import gc
import re
import warnings
from IPython.display import display

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay

# rubin_sim-related packages
import rubin_sim.phot_utils as pt
from rubin_sim.phot_utils import Bandpass
import syseng_throughputs as st
from rubin_sim.data import get_data_dir

# Astropy-related packages
from astropy import units as u
from astropy.io import fits
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.stats import sigma_clipped_stats, mad_std


# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

# Use Rubin standardized colors/symbols/linestyles for u,g,r,i,z,y
from lsst.utils.plotting import (get_multiband_plot_colors,
                                 get_multiband_plot_symbols,
                                 get_multiband_plot_linestyles)

# Set filter warnings to "ignore" to avoid a lot of "logorrhea" to the screen:
warnings.filterwarnings("ignore")

### 1.2 Include user input

In [ ]:
# Which repo, collection, instrument, and skymap to use.
# See https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.-ComCam
# and https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

#instrument = 'LSSTComCam'
#repo = '/repo/dp1'
#collections = 'LSSTComCam/DP1'
#skymap_name = 'lsst_cells_v1'
#day_obs_start = 20241101
#day_obs_end = 20241231

instrument = 'LSSTCam'
repo = '/repo/main'
collections = 'LSSTCam/runs/DRP/20250604_20250921/w_2025_39/DM-52645'
#collections='LSSTCam/runs/DRP/20250421_20250921/w_2025_41/DM-52836' (missing processed ECDFS z-band???)
skymap_name = 'lsst_cells_v1'
day_obs_start = 20250401
day_obs_end = 20251230

# Set environment variable to point to location of the rubin_sim_data 
#  (per Lynne Jones' Slack message on the #sciunit-photo-calib channel from 26 Nov 2024):
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# calspec filename
calspec_filename = "./mag_CalSpec.csv"
#Star_Name = Star_Name
#Star_Name = "WDFS1930-52"
#Star_Name = "NGC6681-1"
#Star_Name = "WDFS1514+00"
#Star_Name = "WDFS1206-27"
#Star_Name = "VB8"
#Star_Name = "WDFS1055-36"
#Star_Name = "WDFS1837-70"
Star_Name = "C26202"
sed_key = 'stiswfcnic_007'
#Star_Name = "WDFS2317-29"
#Star_Name = "WDFS1434-28"
#Star_Name = "WDFS1535-77"

# location of the CalSpec SED FITS files...
calspec_sed_path = "~/Downloads"
calspec_sed_path = os.path.expanduser(calspec_sed_path)
print(calspec_sed_path)

# List of filters to examine
flist = ['u','g','r','i','z','y']

# Plot symbol colors to use for ugrizy
plot_filter_colors_white_background = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00', 'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

# Variables controlling output...
verbose = 3         # verbose = 0, 1, 2, 3, ...  Higher numbers mean more output.
outputCSV = True    # output CSV files

# There was a major change in the DRP pipeline starting with w_2025_05.
# See:  https://rubin-obs.slack.com/archives/C07TXQUAXUZ/p1738795935921129
post_w_2025_04 = True

### 1.3 Define useful classes and functions

In [ ]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [ ]:
# DEPRECATED, since we are no longer useing icSrc for the _instFlux'es
#  (but keep this function around for the time being, just in case...)

# Cartesian x,y match with error (per Claude-3.5-Sonnet)

def cartesianXYMatchWithError(df1, xcol1, ycol1, df2, xcol2, ycol2, sep_limit=1.0, allMatches=True):
    
    import numpy as np
    from scipy.spatial import cKDTree
    import pandas as pd

    # Create KD-tree for efficient spatial searching
    tree = cKDTree(df2[[xcol2, ycol2]])

    # Find nearest neighbors within sep_limit
    separations, indices = tree.query(df1[[xcol1, ycol1]],
                                  distance_upper_bound=sep_limit)

    # Create mask for valid matches (separations less than sep_limit)
    valid_matches = separations < sep_limit

    # Create merged dataframe using only valid matches
    merged_df = pd.concat([
        df1[valid_matches].reset_index(drop=True),
        df2.iloc[indices[valid_matches]].reset_index(drop=True)
        ], axis=1)

    # If you want to keep track of the match separations
    merged_df['separation'] = separations[valid_matches]

    # If you want to keep just the best match, sort by separation 
    # and keep first occurrence of each df2 index
    if allMatches != True:
        merged_df = merged_df.sort_values('separation').drop_duplicates(
            subset=df2.columns, keep='first'
        )

    return merged_df



### 1.4 Instantiate the butler and registry

In [ ]:
butler = dafButler.Butler(repo, collections=collections)
registry = butler.registry

### 1.5 Read calspec file and convert to a python dictionary

In [ ]:
# Read calspec file and convert to a python dictionary

# Read CSV into a DataFrame
df = pd.read_csv(calspec_filename)

# Convert to list of dictionaries
data = df.to_dict(orient="records")

# Or: dictionary of dictionaries keyed by Star_Name
data_by_star = df.set_index("Star_Name").to_dict(orient="index")

print(data_by_star[Star_Name])

raDeg = data_by_star[Star_Name]["raDeg"]
decDeg = data_by_star[Star_Name]["decDeg"]

# Grab the row dictionary for this star
row = data_by_star[Star_Name]

# Build dictionary of file names
sedfile_dict = {}

# Loop over the last three columns
for col in ["STIS", "Model"]:
    val = row[col]
    if pd.notna(val) and val != "":
        # strip leading underscore if present
        key = val.strip("_")
        filename = f"{row['Name']}_{key}.fits"
        sedfile_dict[key] = os.path.join(calspec_sed_path, filename)

print(sedfile_dict)



## 2. Estimate expected counts for airmasses X=1.0 to 2.5

### 2.1 Build the hardware and system for ugrizy for Cerro Pachon for airmasses X=1.0-2.5 in steps of 0.1 airmass

In [ ]:
# From https://github.com/lsst-pst/syseng_throughputs/blob/main/notebooks/InterpolateZeropoint.ipynb

defaultDirs = st.setDefaultDirs()
if instrument == "LSSTComCam":
    #Change detectors from (default) LSST to ComCam (ITL CCDs)
    defaultDirs['detector'] = defaultDirs['detector'].replace('/joint_minimum', '/itl')

airmasses = np.arange(1.0, 2.6, 0.1).round(2)

system = {}
for x in airmasses:
    #atmos files temporarily inaccessible from /sdf/data/rubin/shared/rubin_sim_data/throughputs/atmos;
    # using a temporary solution here:
    #atmosDir = '/home/d/dltucker/DATA/rubin_sim_data_throughputs/throughputs/atmos'
    #atmos = st.readAtmosphere(atmosDir, atmosFile=f'atmos_{x*10 :.0f}_aerosol.dat')
    atmos = st.readAtmosphere(os.path.join(get_data_dir(), 'throughputs', 'atmos'), atmosFile=f'atmos_{x*10 :.0f}_aerosol.dat')
    h, s = st.buildHardwareAndSystem(defaultDirs, addLosses=True,  atmosphereOverride=atmos)
    system[x] = s
hardware = h


### 2.2 Plot filter passbands (without the atmospheric component) and the atmospheric transmission for airmasses 1.0, 1.2, 2.0

In [ ]:
# From https://github.com/lsst-pst/syseng_throughputs/blob/main/notebooks/InterpolateZeropoint.ipynb

# Plot only if verbosity level is higher than 2...
if verbose > 2:
    
    colors = plot_filter_colors_white_background
    for f in flist:
        plt.plot(hardware[f].wavelen, hardware[f].sb, color=colors[f], linestyle=':')
    for x in [1.0, 1.2, 2.0]:
        #atmos files temporarily inaccessible from /sdf/data/rubin/shared/rubin_sim_data/throughputs/atmos;
        # using a temporary solution here:
        #atmosDir = '/home/d/dltucker/DATA/rubin_sim_data_throughputs/throughputs/atmos'
        #atmos = st.readAtmosphere(atmosDir, atmosFile=f'atmos_{x*10 :.0f}_aerosol.dat')
        atmos = st.readAtmosphere(os.path.join(get_data_dir(), 'throughputs', 'atmos'), atmosFile=f'atmos_{x*10 :.0f}_aerosol.dat')
        plt.plot(atmos.wavelen, atmos.sb, linestyle='-')
    plt.ylim(0, 1)
    plt.xlim(300, 1100)
    plt.xlabel("Wavelength (nm)")


### 2.3 Read in the CalSpec SED file and translate it into `rubin_sim` format 

In [ ]:
# Read the sedfile

print(sed_key, sedfile_dict[sed_key])
    
# Read the SED file associated with this SED
sedfile = sedfile_dict[sed_key]
seddata = fits.getdata(sedfile)

# Transform the SED data into rubin_sim format
wavelen = seddata['WAVELENGTH'] * u.angstrom.to(u.nanometer) # This is in angstroms - need in nanometers
flambda = seddata['FLUX'] / (u.angstrom.to(u.nanometer)) # this is in erg/sec/cm^^2/ang but we want /nm     
sed = pt.Sed(wavelen=wavelen, flambda=flambda)

In [ ]:
# Plot star spectrum (in normalized photon flux) with filter passbands

filter_colors = get_multiband_plot_colors()
filter_symbols = get_multiband_plot_symbols()
filter_linestyles = get_multiband_plot_linestyles()

x = 1.2
#atmos = st.readAtmosphere(atmosDir, atmosFile=f'atmos_{x*10 :.0f}_aerosol.dat')
atmos = st.readAtmosphere(os.path.join(get_data_dir(), 'throughputs', 'atmos'), atmosFile=f'atmos_{x*10 :.0f}_aerosol.dat')

# Create empty list to store legend handles
legend_handles = []

for f in flist:
    # Store the line handle when plotting
    line, = plt.plot(hardware[f].wavelen, hardware[f].sb*atmos.sb, 
                    color=filter_colors[f], 
                    linestyle=filter_linestyles[f],
                    label=f'{f}-band')  # Add label for each filter
    legend_handles.append(line)

# Constants
h = 6.626e-34  # Planck constant in J*s
c = 2.998e8    # Speed of light in m/s

# Calculate photon flux
wavelen_m = wavelen * 1e-9
flambda_joules = flambda * 1e-7
fphoton = flambda_joules * wavelen_m / (h * c)

# Store the stellar flux line handle
label = """%s spectrum""" % (Star_Name)
stellar_line, = plt.plot(wavelen, fphoton/max(fphoton), 
                        color='darkgrey', 
                        linestyle='-',
                        label=label)
legend_handles.append(stellar_line)

plt.xlim([300., 1100.])
plt.ylim([0.0, 1.0])
plt.xlabel("Wavelength (nm)")
plt.ylabel("Bandpass Total Throughput;  Stellar Photon Flux (normalized)")

# Add the legend
plt.legend(handles=legend_handles, loc='upper right')

### 2.4 Define the photometric parameters to use.

In [ ]:
phot_params = pt.PhotometricParameters(exptime=30, nexp=1, gain=1.0)


### 2.5 Calculate the expected counts for CalSpec star for the given photometric parameters over the airmass range of X=1.0-2.5

First, calculate the expected counts using the standard 30-second exposure time...

In [ ]:
counts = {}
for f in flist:
    counts[f] = []
    for x in airmasses:
        counts[f].append(sed.calc_adu(system[x][f], phot_params))
    counts[f] = np.array(counts[f])


In [ ]:
df_counts = pd.DataFrame(counts, index=airmasses)

if verbose > 0:
    display(df_counts)

### 2.6 Output results to CSV file

In [ ]:
if outputCSV:
    sedfile_base = os.path.splitext(os.path.basename(sedfile))[0]
    outputFile = f"{instrument}.{sedfile_base}.expected_counts.csv"
    print(f"Outputting expected counts to {outputFile}")
    df_counts.to_csv(outputFile, index=True)  #  Here, we want to keep the index for the DataFrame, which, in this case, is the airmass

### 2.7 Create tophat passbands covering the same range of wavelengths of the original filter passbands

In [ ]:
from copy import deepcopy
x=1.2
system_tophat = {}
for f in flist:
    system_tophat[f] = deepcopy(system[x][f])
    system_tophat[f].sb = np.where(system_tophat[f].sb >= 0.001, 1.0, 0.0)


Plot both the original (X=1.2) passbands and the tophat passbands together.

In [ ]:
filter_colors = get_multiband_plot_colors()
filter_symbols = get_multiband_plot_symbols()
filter_linestyles = get_multiband_plot_linestyles()

x = 1.2

# Create empty list to store legend handles
legend_handles = []

for f in flist:
    line, = plt.plot(system[x][f].wavelen, system[x][f].sb, 
                    color=filter_colors[f], 
                    linestyle=filter_linestyles[f],
                    label=f'{f}-band original')  # Add label for each filter
    legend_handles.append(line)

    line, = plt.plot(system_tophat[f].wavelen, system_tophat[f].sb, 
                    color=filter_colors[f], 
                    linestyle='-',
                    label=f'{f}-band tophat')  # Add label for each filter
    legend_handles.append(line)


plt.xlim([300., 1100.])
plt.ylim([0.0, 1.1])
plt.xlabel("Wavelength (nm)")
plt.ylabel("Bandpass Total Throughput")

# Add the legend
plt.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()  # adjust layout so labels don’t get cut off

### 2.8 Calculate the expected counts for CalSpec star for the given photometric parameters for the tophat passbands

In [ ]:
counts_tophat = {}
for f in flist:
    counts_tophat[f] = float(sed.calc_adu(system_tophat[f], phot_params))


In [ ]:
counts_tophat

## 3. Query USDF Butler for exposures

### 3.1 Create query

In [ ]:
# Taken from vv-team-notebooks/reports/TargetReport.ipynb

# Build WHERE clause

where = f"instrument='{instrument}' AND day_obs>={day_obs_start} AND day_obs<={day_obs_end}"

# Query all exposure records in one go
results = list(registry.queryDimensionRecords("exposure", where=where))



### 3.2 Check that there are results; stop execution if there are none

In [ ]:
# Taken from vv-team-notebooks/reports/TargetReport.ipynb

# Stop executing if there are no results returned:

n_results = len(results)
if n_results == 0:
    raise StopExecution
else:
    print(f"There are {n_results} results returned from querying the butler for instrument {instrument} "
          f"between dates {day_obs_start} and {day_obs_end} (inclusive).")


### 3.3 Extract all rows into a list of tuples

In [ ]:
# Taken from vv-team-notebooks/reports/TargetReport.ipynb

rows = []
for info in results:
    try:
        rows.append((
            info.id,
            info.obs_id,
            info.day_obs,
            info.seq_num,
            info.timespan.begin,
            info.timespan.end,
            info.observation_type,
            info.observation_reason,
            info.target_name,
            info.physical_filter,
            info.zenith_angle,
            info.exposure_time,
            info.tracking_ra,
            info.tracking_dec,
            info.sky_angle,
            info.azimuth,
            info.zenith_angle,
            info.science_program
        ))
    except Exception as e:
        # Fallback values if timespan is missing or broken
        rows.append((
            info.id,
            info.obs_id,
            info.day_obs,
            info.seq_num,
            pd.to_datetime("2021-01-01 00:00:00.00"),
            pd.to_datetime("2051-01-01 00:00:00.00"),
            info.observation_type,
            info.observation_reason,
            info.target_name,
            info.physical_filter,
            info.zenith_angle,
            info.exposure_time,
            info.tracking_ra,
            info.tracking_dec,
            info.sky_angle,
            info.azimuth,
            info.zenith_angle,
            info.science_program
        ))


### 3.4 Build DataFrame in one shot

In [ ]:
# Taken from vv-team-notebooks/reports/TargetReport.ipynb

df_exp = pd.DataFrame(rows, columns=[
    'id', 'obs_id', 'day_obs', 'seq_num',
    'time_start', 'time_end', 'type', 'reason',
    'target', 'filter', 'zenith_angle',
    'expos', 'ra', 'dec', 'skyangle',
    'azimuth', 'zenith', 'science_program'
])

# Display current version of df_exp
#df_exp

### 3.5 Clean up DataFrame

In [ ]:
# Taken from vv-team-notebooks/reports/TargetReport.ipynb

# Compute JD and MJD vectorized
t_start = Time(df_exp['time_start'].tolist(), scale='tai')
df_exp['jd'] = t_start.jd
df_exp['mjd'] = t_start.mjd


In [ ]:
# Taken from vv-team-notebooks/reports/TargetReport.ipynb

# Re-cast the `id`, `day_obs`, and `seq_num` rows as `int`'s:
df_exp = df_exp.astype({"id": int,'day_obs': int,'seq_num':int})

In [ ]:
# Taken from vv-team-notebooks/reports/TargetReport.ipynb

# Replace `NaN`'s in the `ra` and `dec` columns with zero.  
# (`NaN`'s in `ra`, `dec` wreak havoc for the healpix tools defined in Section 1.2 above.) 
# ***(Maybe no longer necessary?)***

df_exp['ra'] = df_exp['ra'].fillna(0)
df_exp['dec'] = df_exp['dec'].fillna(0)

In [ ]:
df_exp

### 3.6 Add airmass to DataFrame

In [ ]:
# Add an airmass to df_exp...

df_exp['airmass'] = np.round(1./np.cos(np.deg2rad(df_exp['zenith_angle'])), decimals=3)

In [ ]:
# Printout zenith angle and airmass if verbosity level is greater than 1...
if verbose > 1:
    display(df_exp[['zenith_angle','airmass']])

### 3.7 Extract just "science" exposures

In [ ]:
# Create a `DataFrame` containing just the science exposures:
df_sci = df_exp[df_exp.type == 'science']

In [ ]:
# Look at columns for the (exposure/visit) id, zenith_angle, and airmass, 
#  but only if verbosity level is greater than 1:
if verbose > 1:
    display(df_sci[['id', 'zenith_angle','airmass']])

### 3.8 Remove any exposures in the "bad visit" list

#### 3.8.1 Read in "bad visit" list

In [ ]:
if instrument == "LSSTComCam":
    df_bad_visits=Table.read("https://raw.githubusercontent.com/lsst-dm/excluded_visits/refs/heads/main/LSSTComCam/bad.ecsv").to_pandas()
    #df_bad_visits.rename(columns={'exposure': 'visit'}, inplace=True)
else: 
    df_bad_visits=Table.read("https://raw.githubusercontent.com/lsst-dm/excluded_visits/refs/heads/main/LSSTCam/bad.ecsv").to_pandas()

# Look at bad visits table, but only if verbosity level is greater than 0:
if verbose > 0:
    display(df_bad_visits)

#### 3.8.2 Remove from df_sci any exposures found in df_bad_visits

In [ ]:
df_sci = df_sci[~df_sci['id'].isin(df_bad_visits['exposure'])]


In [ ]:
# Look at columns for the (exposure/visit) id, zenith_angle, and airmass, 
#  but only if verbosity level is greater than 0:
if verbose > 0:
    display(df_sci[['id', 'zenith_angle','airmass']])

### 3.9 Save results as a CSV file

In [ ]:
if outputCSV:
    outputFile = f"{instrument}.sci_exps.{day_obs_start}-{day_obs_end}.csv"
    print(f"Outputting exposures to {outputFile}")
    df_sci.to_csv(outputFile, index=False)

### 3.10 Create a Pandas DataFrame from df_sci that just contains the visit id, exposure time, zenith angle, and airmass

In [ ]:
df_sci_airmass = df_sci[['id', 'expos', 'zenith_angle','airmass']].copy(deep=True)
df_sci_airmass.reset_index(drop=True, inplace=True)

# Look at pandas dataframe, but only if verbosity level is greater than 0:
if verbose > 0:
    display(df_sci_airmass)

## 4. Query USDF Butler for measurements of CalSpec star

### 4.1 Find the `dataId`'s for all `visit_image`'s in this repo/collection that overlap the sky position of CalSpec star

In [ ]:
datasetRefs = butler.query_datasets("visit_image", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

for i, ref in enumerate(datasetRefs):    
    print(i, ref.dataId)
    if ((verbose < 2) & (i >= 10)): 
        print("...")
        break


print(f"\nFound {len(datasetRefs)} visit_images")

### 4.2 Create a pandas Dataframe containing the `source2` info for all these `visit_image`'s

#### 4.2.1 Loop over the `datasetRefs` again, grabbing the contents of the `source2` table for each `ref` and combining into all into one big pandas DataFrame.  

In [ ]:
# Reference CALSPEC star coordinates
ref_coord = SkyCoord(ra=raDeg*u.degree, dec=decDeg*u.degree)

src_list = []

for i, ref in enumerate(datasetRefs):
    dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
    src = butler.get('source2', dataId=dataId).to_pandas()
#    src = butler.get('recalibrated_star_detector', dataId=dataId).to_pandas()
# NOTE - source2 has more matches and gives a slightly different offset to recalibrated - which is going away soon (less than 2 mmag)

    # Apply "good measurement" mask immediately
    mask = (~src.pixelFlags_bad) & (~src.pixelFlags_saturated) & \
           (~src.extendedness_flag)
    src_cleaned = src[mask]

    # Compute separations to CALSPEC star
    df_coords = SkyCoord(ra=src_cleaned['ra'].values*u.degree,
                         dec=src_cleaned['dec'].values*u.degree)
    separations = ref_coord.separation(df_coords)

    # Keep only sources within 60 arcsec (for median aperture corrections later)
    mask_sep = separations < 60.0*u.arcsec
    nearby = src_cleaned[mask_sep].copy()
    nearby['separation_calspec'] = separations[mask_sep].arcsec
    
    if not nearby.empty:
        #best = nearby.sort_values('separation_calspec').iloc[[0]]
        #src_list.append(best)
        src_list.append(nearby)
        if ((verbose >= 2) | (i < 10)): 
            #print(f"{i} Visit {ref.dataId['visit']}, Detector {ref.dataId['detector']}: "
            #      f"Found {len(best)} candidate matches.")
            print(f"{i} Visit {ref.dataId['visit']}, Detector {ref.dataId['detector']}: "
                  f"Found {len(nearby)} candidate matches.")
        if ((verbose < 2) & (i == 10)): 
            print("...")
            
# Concatenate only the small filtered tables
if src_list:
    src_all = pd.concat(src_list, ignore_index=True)
    print(f"\nTotal combined catalog contains {len(src_all)} candidate sources.")
else:
    print("No matches found within 60 arcsec.")


In [ ]:
# Show resulting pandas dataframe, but only if verbosity level is greater than 1:
if verbose > 1:
    display(src_all)

#### 4.2.2 Add exposure time, zenith distance, and airmass to src_all

In [ ]:
src_all_tmp = pd.merge(src_all, df_sci_airmass, left_on='visit', right_on='id')
src_all_tmp.drop('id', axis=1, inplace=True)
# Remove any rows for which airmass is a NaN
src_all_tmp.dropna(subset=['airmass'], inplace=True)
src_all = src_all_tmp

# Show resulting pandas dataframe, but only if verbosity level is greater than 0:
if verbose > 0:
    display(src_all)

#### 4.2.3 Add fractional flux error `apFlux_12_0_instFracFluxErr` to src_all

In [ ]:
src_all['apFlux_12_0_instFracFluxErr'] = src_all['apFlux_12_0_instFluxErr']/src_all['apFlux_12_0_instFlux']


In [ ]:
if verbose > 2:
    for c in src_all.columns:
        print(c)

In [ ]:
if verbose > 2:
    for c in src_all.filter(regex='_inst').columns:
        print(c)


#### 4.2.3 Save `src_all` as a CSV file

In [ ]:
if outputCSV:
    outputFile = f"{instrument}.{Star_Name}.source2.csv"
    print(f"Outputting observed source2 table results for {Star_Name} to {outputFile}")
    src_all.to_csv(outputFile, index=False)

## 5 Calculate psf to total flux aperture magnitudes on a per-visit basis

### 5.1 For historical reasons, define df_match to be src_all

In [ ]:
# A TEMPORARY KLUDGE!
df_match = src_all.copy()

In [ ]:
df_match

### 5.2 Calculate values for per-visit `local_photoCalib` and use these values to back-calculate `psfFlux_inst` from `psfFlux`

***We do this since neither `psfFlux_inst` nor `local_photoCalib` seems to be in `source2`.***

#### 5.2.1 Calculate values for per-visit `local_photoCalib`

We use the 12-, 17-, 35-, and 50-pixel aperture fluxes. 

(Note that the values for `local_photoCalib` calculated from each of these aperture fluxes are (nearly) identical, with any differences typcially being at the 0.00001 level or smaller, most likely due to the clipping process in the calculation.  This is a good sign that `local_photoCalib` is well calculated.)

In [ ]:
# Kudos to CoPilot+GPT5

# Define the apertures you want to process
apertures = {
    "12_0": ("ap12Flux", "apFlux_12_0_instFlux", "apFlux_12_0_flag"),
    "17_0": ("ap17Flux", "apFlux_17_0_instFlux", "apFlux_17_0_flag"),
    "35_0": ("ap35Flux", "apFlux_35_0_instFlux", "apFlux_35_0_flag"),
    "50_0": ("ap50Flux", "apFlux_50_0_instFlux", "apFlux_50_0_flag"),
}

# Dictionary to hold median calibration DataFrames
median_dfs = []

for ap_label, (flux_col, instflux_col, flag_col) in apertures.items():

    # Create the local calibration ratio column
    calib_col = f"local_photoCalib_{ap_label}"
    df_match[calib_col] = df_match[flux_col] / df_match[instflux_col]

    # Create mask
    #  Create a mask to cull sources with "bad" measurements.
    mask1 = (~df_match.pixelFlags_bad) & \
            (~df_match.pixelFlags_saturated) & \
            (~df_match.extendedness_flag) & \
            (~df_match[flag_col])  
    #  Create an another mask to cull sources that are too faint or (possibly) too bright.
    flux_min = df_match[mask1][instflux_col].quantile(0.75)
    flux_max = df_match[mask1][instflux_col].quantile(0.95)
    # Combine masks
    mask = mask1 & \
        (df_match[instflux_col] >= flux_min) & \
        (df_match[instflux_col] < flux_max) & \
        (df_match.apFlux_12_0_instFracFluxErr >= 0.00) & \
        (df_match.apFlux_12_0_instFracFluxErr < 0.02)


    # Compute median per visit, ignoring NaNs
    median_series = (
        df_match[mask]
        .groupby("visit")[calib_col]
        .agg(lambda x: np.nanmedian(x))
    )

    # Convert to DataFrame and rename column
    median_df = median_series.reset_index().rename(
        columns={calib_col: calib_col}
    )
    median_dfs.append(median_df)

# Merge all median DataFrames on 'visit'
from functools import reduce
df_median_local_photoCalib = reduce(
    lambda left, right: pd.merge(left, right, on="visit", how="outer"),
    median_dfs
)

# Optional: inspect
if verbose > 1:
    display(df_median_local_photoCalib)


In [ ]:
# Note:  the above procedure also appends these local_photoCalib_* columns to df_match:
if verbose > 2:
    display(df_match)

#### 5.2.2 Back-calculate psfFlux_inst from psfFlux

In [ ]:
# Use local_photoCalib_12_0 for the conversion
df_match['psfFlux_instFlux'] = df_match['psfFlux']/df_match['local_photoCalib_12_0']

# Optional: inspect
if verbose > 2:
    display(df_match)


### 5.3 Create dataframe containing the visit-by-visit median psf-to-total flux aperture corrections.

In [ ]:
# Create a column containing the aperture-to-total flux aperture correction for each individual source.

#  We will use 'psfFlux_instFlux' as our primary instrumental flux measurement.
#  We will take 'apFlux_50_0_instFlux' as the total flux.
df_match['apCorrTot'] = df_match['apFlux_50_0_instFlux'] / df_match['psfFlux_instFlux']

# Create a mask to cull sources with "bad" measurements.
mask1 = (~df_match.pixelFlags_bad) & \
        (~df_match.pixelFlags_saturated) & \
        (~df_match.extendedness_flag) & \
        (~df_match.apFlux_12_0_flag) & \
        (~df_match.apFlux_50_0_flag)  

# Create an another mask to cull sources that are too faint or (possibly) too bright.
flux_min = df_match[mask1]['psfFlux_instFlux'].quantile(0.75)
flux_max = df_match[mask1]['psfFlux_instFlux'].quantile(0.95)
mask = mask1 & \
        (df_match.psfFlux_instFlux >= flux_min) & \
        (df_match.psfFlux_instFlux < flux_max)  & \
        (df_match.apFlux_12_0_instFracFluxErr >= 0.00) & \
        (df_match.apFlux_12_0_instFracFluxErr < 0.01)

# Calculate median ratio per visit, ignoring NaNs
median_apCorrTots = df_match[mask].groupby('visit')['apCorrTot'].agg(lambda x: np.nanmedian(x))

# Create a pandas DataFrame out of this pandas Series
df_median_apCorrTots = median_apCorrTots.reset_index()

# Rename `apCorrTot` to `apCorrTot_median` in df_median_apCorrTots
df_median_apCorrTots.rename(columns={'apCorrTot': 'apCorrTot_median'}, inplace=True)

## Remove the original apCorrTot column from df_match
#df_match.drop('apCorrTot', axis=1, inplace=True)

# Show the dataframe of median apCorrTots by visit id, 
#  but only if verbosity level is greater than 1:
if verbose > 1:
    display(df_median_apCorrTots)

### 5.4 Add the visit-by-visit median aperture corrections to the `df_match`pandas DataFrame

In [ ]:
df_match = df_match.merge(df_median_apCorrTots, on='visit')

In [ ]:
# Display result sorted in ascending order of visit (primarily) and RA (secondarily), 
#  but only if verbosity level is greater than 0:
if verbose > 0:
    display(df_match.sort_values(by=['visit', 'ra']))

## 6. Extract the rows containing CalSpec star from the df_match catalog

In [ ]:
# Based on code retrieved from Claude-3.5-Sonnet

# Create a mask to cull sources with "bad" measurements.
mask1 = (~df_match.pixelFlags_bad) & \
        (~df_match.pixelFlags_saturated) & \
        (~df_match.extendedness_flag) & \
        (~df_match.apFlux_12_0_flag) & \
        (~df_match.psfFlux_flag)  

mask = mask1 & \
        (df_match.apFlux_12_0_instFracFluxErr >= 0.00) & \
        (df_match.apFlux_12_0_instFracFluxErr < 0.01)


# Apply mask, keeping only the "good" measurements of `df_match`
df_match_cleaned = df_match[mask]

# Create SkyCoord object for the coordinates of CalSpec star
ref_coord = SkyCoord(ra=raDeg*u.degree, dec=decDeg*u.degree)

# Create SkyCoord object for all points in the dataframe
df_coords = SkyCoord(ra=df_match_cleaned['ra'].values*u.degree, 
                     dec=df_match_cleaned['dec'].values*u.degree)

# Calculate separations
separations = ref_coord.separation(df_coords)

# Create mask for points within 3.0 arcseconds
mask_sep = separations < 3.0*u.arcsec

# Get filtered dataframe
nearby_good_df = df_match_cleaned[mask_sep]

# If you want to include the separations in the result
orig_columns = nearby_good_df.columns
nearby_good_df = df_match_cleaned[mask_sep].copy()
nearby_good_df['separation_calspec'] = separations[mask_sep].arcsec

# Find (and keep) the closet match within the match radius
best_df = nearby_good_df.sort_values('separation_calspec').drop_duplicates(subset=orig_columns, keep='first')


In [ ]:
# Display the resulting table, but only if verbosity level is greater than 1:
if verbose > 1:
    display(best_df)

In [ ]:
# Display only the most relevant columns of the resulting table, 
# but only if verbosity level is greater than 0:
if verbose > 0:
    #display(best_df[['visit', 'band', 'expos', 'airmass', 'apFlux_12_0_instFlux', 'apCorrTot_median']])
    display(best_df[['visit', 'band', 'expos', 'airmass', 'psfFlux_instFlux', 'apCorrTot_median']])

In [ ]:
#for colname in best_df.columns:
#    print(colname)

## 7. Calculate the ratio of observed to expected throughputs for instrument based on CalSpec star


### 7.1 Add a column to `best_df` containing the expected counts for CalSpec star based on the contents of `df_counts` created earlier

We will use the `interp1d` interpolation function from the `scipy.interpolate` package to perform linear interpolations between the airmasses listed in `df_counts`.

In [ ]:
# Based on code retrieved from Claude-3.5-Sonnet

from scipy.interpolate import interp1d

# Create a dictionary to store interpolation functions for each band
interpolators = {}
for band in flist:
    interpolators[band] = interp1d(df_counts.index, 
                                 df_counts[band], 
                                 kind='linear',
                                 bounds_error=False,    # Return nan for out of bounds
                                 fill_value=np.nan)

# Create new column with interpolated values
best_df['total_counts_expected'] = best_df.apply(
    lambda row: interpolators[row['band']](row['airmass']), 
    axis=1
)

# You can check the results if verbosity level is greater than 0):
if verbose > 0:
    display(best_df[['visit', 'band', 'airmass', 'total_counts_expected']])

# Optional: Check for any NaN values (would indicate airmass outside interpolation range)
nan_matches = best_df[best_df['total_counts_expected'].isna()]
if len(nan_matches) > 0:
    print("\nRows with no matches (airmass out of range):")
    print(nan_matches[['visit', 'band', 'airmass']])

### 7.2 Add columns to `best_df` containing the total counts observed and the ratio of total counts observed to total counts expected


In [ ]:
best_df['total_counts_observed'] = best_df['apCorrTot_median'] * best_df['psfFlux_instFlux']
# We will rescale total_counts_observed to a 30-second exposure for those 
#  (u-band) visits that have a 38-second exposure time:
best_df['total_counts_observed'] = (30./best_df['expos'])*best_df['total_counts_observed']
best_df['ratio_obs_exp'] = best_df['total_counts_observed'] / best_df['total_counts_expected']

Let's look at them...

In [ ]:
# Set pandas to show all rows (but only if verbosity level is greater than 1)...
if verbose > 1:
    pd.set_option("display.max_rows", None)

In [ ]:
# Output to screen the most relevant columns for all rows, 
#  but only if verbosity level is greater than 0...
if verbose > 0:
    display(best_df[['visit', 'band', 'airmass', 'expos', 'psfFlux_instFlux', 'apCorrTot_median', 'total_counts_observed', 'total_counts_expected', 'ratio_obs_exp']])

In [ ]:
# Reset pandas to its default maximum rows to print to screen
# (if it had been reset earlier due to verbosity level greater than 1)...
if verbose > 1:
    pd.reset_option("display.max_rows")

### 7.3 Plot a histogram of the ratio of total counts observed to total counts for each passband

In [ ]:
# Based on code retrieved from Claude-3.5-Sonnet and Poe.com Assistant

# Set up the plot
plt.figure(figsize=(10, 6))

# Define colors and transparency for each band
colors = plot_filter_colors_white_background
alpha = 1.0   # transparency level
linewidth = 3 # linewidth for the step histogram lines 

# Define bins.  Here, we want to look around ratio=1.00+/-0.20 in steps of 0.01
#bins = np.arange(0.80, 1.20, 0.01)
#bins = np.arange(0.00, 2.00, 0.01)
bins = np.arange(0.00, 2.00, 0.02)

# Plot histogram for each band
for band in flist:
    band_data = best_df[best_df['band'] == band]['ratio_obs_exp']
    if len(band_data) > 0:  # only plot if we have data for this band
        #plt.hist(band_data, bins=bins, alpha=alpha, histtype='step', linewidth=linewidth, 
        #        label=f'band {band}', color=colors[band],
        #        density=False)  # density=True normalizes the area
        #plt.hist(band_data, bins=bins, alpha=alpha, histtype='step', linewidth=linewidth, 
        #        label=f'band {band}', color=filter_colors[band],
        #        linestyle=filter_linestyles[band], 
        #        density=False)  # density=True normalizes the area
        plt.hist(band_data, bins=bins, alpha=alpha, histtype='step', linewidth=linewidth, 
                label=f'band {band}', color=filter_colors[band],
                density=False)  # density=True normalizes the area


plt.xlabel('Ratio (Observed Counts/Expected Counts)')
plt.ylabel('Number')
#plt.xlim([0.75, 1.15])
plt.xlim([0.75, 1.50])
#plt.xlim([0.00, 2.00])

plot_title = """Distribution of Observed/Expected Total Counts Ratio by Band for %s""" % (Star_Name)
plt.title(plot_title)
plt.legend()
plt.grid(True, alpha=0.3)

# Optional: adjust layout to prevent label clipping
plt.tight_layout()

plt.show()



### 7.3 Print summary statistics for each band

Let's output a nice table...

In [ ]:
# Based on code retrieved from Claude-3.5-Sonnet and from CoPilot+GPT5

stats_data = []

for band in flist:
    # Extract and clean the data
    band_data = best_df.loc[best_df['band'] == band, 'ratio_obs_exp']
    band_data = pd.to_numeric(band_data, errors='coerce')  # force numeric
    band_data = band_data.replace([np.inf, -np.inf], np.nan).dropna().values.astype(float)

    if len(band_data) > 0:
        # --- Option A: sigma-clipped stats ---
        mean_clip, median_clip, std_clip = sigma_clipped_stats(band_data, sigma=3.0, maxiters=5)
        stderr_clip = std_clip / np.sqrt(len(band_data))

        # --- Option B: robust median/MAD ---
        median_robust = np.median(band_data)
        std_robust = mad_std(band_data)
        stderr_robust = std_robust / np.sqrt(len(band_data))

        stats_data.append({
            'band': band,
            'n_band': len(band_data),
            'Mean (clip)': f"{mean_clip:.3f}",
            'Median (clip)': f"{median_clip:.3f}",
            'Std (clip)': f"{std_clip:.3f}",
            'StdErr (clip)': f"{stderr_clip:.3f}",
            'Median (robust)': f"{median_robust:.3f}",
            'Std (robust)': f"{std_robust:.3f}",
            'StdErr (robust)': f"{stderr_robust:.3f}"
        })
    else:
        stats_data.append({
            'band': band,
            'n_band': 0,
            'Mean (clip)': 'N/A',
            'Median (clip)': 'N/A',
            'Std (clip)': 'N/A',
            'StdErr (clip)': 'N/A',
            'Median (robust)': 'N/A',
            'Std (robust)': 'N/A',
            'StdErr (robust)': 'N/A'
        })

# Create DataFrame
stats_df = pd.DataFrame(stats_data)

# Display the full table
display(stats_df)
#print(stats_df.to_string(index=False))

# Display a truncated table
display(stats_df[['band', 'n_band', 'Mean (clip)', 'Median (clip)', 'StdErr (clip)']])


## 8. Calculate the absolute throughputs for instrument based on CalSpec star


We do this by comparing the observed throughputs from the real filters to the expected throughputs from the tophat filters.

### 8.1 Add columns to `best_df` containing the total expected counts from the tophat filters and the ratio of total counts observed from the real filters to total counts expected from the tophat filters


In [ ]:
df_counts_tophat = pd.Series(counts_tophat, name="total_counts_expected_tophat")
best_df = best_df.join(df_counts_tophat, on="band")
best_df['abs_throughput'] = best_df['total_counts_observed'] / best_df['total_counts_expected_tophat']


In [ ]:
# Set pandas to show all rows (but only if verbosity level is greater than 1)...
if verbose > 1:
    pd.set_option("display.max_rows", None)

In [ ]:
# Output to screen the most relevant columns for all rows, 
#  but only if verbosity level is greater than 0...
if verbose > 0:
    display(best_df[['visit', 'band', 'airmass', 'expos', 'total_counts_observed', 'total_counts_expected_tophat', 'abs_throughput']])

In [ ]:
# Reset pandas to its default maximum rows to print to screen
# (if it had been reset earlier due to verbosity level greater than 1)...
if verbose > 1:
    pd.reset_option("display.max_rows")

### 8.2 Plot a histogram and a scatter plot of the absolute throughputs for each passband

In [ ]:
# Based on code retrieved from Claude-3.5-Sonnet and Poe.com Assistant

# Set up the plot
plt.figure(figsize=(10, 6))

# Define colors and transparency for each band
colors = plot_filter_colors_white_background
alpha = 1.0   # transparency level
linewidth = 3 # linewidth for the step histogram lines 

# Define bins.  Here, we want to look around ratio=1.00+/-0.20 in steps of 0.01
#bins = np.arange(0.80, 1.20, 0.01)
#bins = np.arange(0.00, 2.00, 0.01)
bins = np.arange(0.00, 2.00, 0.02)

# Plot histogram for each band
for band in flist:
    band_data = best_df[best_df['band'] == band]['abs_throughput']
    if len(band_data) > 0:  # only plot if we have data for this band
        #plt.hist(band_data, bins=bins, alpha=alpha, histtype='step', linewidth=linewidth, 
        #        label=f'band {band}', color=colors[band],
        #        density=False)  # density=True normalizes the area
        #plt.hist(band_data, bins=bins, alpha=alpha, histtype='step', linewidth=linewidth, 
        #        label=f'band {band}', color=filter_colors[band],
        #        linestyle=filter_linestyles[band], 
        #        density=False)  # density=True normalizes the area
        plt.hist(band_data, bins=bins, alpha=alpha, histtype='step', linewidth=linewidth, 
                label=f'band {band}', color=filter_colors[band],
                density=False)  # density=True normalizes the area


plt.xlabel('Absolute Throughput')
plt.ylabel('Number')
plt.xlim([0.00, 1.00])

plot_title = """Distribution of Estimated Absolute Throughputs by Band using %s""" % (Star_Name)
plt.title(plot_title)
plt.legend()
plt.grid(True, alpha=0.3)

# Optional: adjust layout to prevent label clipping
plt.tight_layout()

plt.show()



In [ ]:
filter_colors = get_multiband_plot_colors()
filter_symbols = get_multiband_plot_symbols()
filter_linestyles = get_multiband_plot_linestyles()

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Loop over filters
for band in flist:
    band_data = best_df[best_df['band'] == band]
    if len(band_data) > 0:
        plt.scatter(
            band_data["airmass"],
            band_data["abs_throughput"],
            label=f"band {band}",
            marker=filter_symbols[band], 
            c=filter_colors[band],
            alpha=0.8
        )

plt.xlabel("Airmass")
plt.ylabel("Passband Absolute Throughput")
plt.title(f"Absolute Throughput vs. Airmass by Band using {Star_Name}")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


### 8.3 Print summary statistics for each band

Let's output a nice table...

In [ ]:
# Based on code retrieved from Claude-3.5-Sonnet and from CoPilot+GPT5

stats_data = []

for band in flist:
    # Extract and clean the data
    band_data = best_df.loc[best_df['band'] == band, 'abs_throughput']
    band_data = pd.to_numeric(band_data, errors='coerce')  # force numeric
    band_data = band_data.replace([np.inf, -np.inf], np.nan).dropna().values.astype(float)

    if len(band_data) > 0:
        # --- Option A: sigma-clipped stats ---
        mean_clip, median_clip, std_clip = sigma_clipped_stats(band_data, sigma=3.0, maxiters=5)
        stderr_clip = std_clip / np.sqrt(len(band_data))

        # --- Option B: robust median/MAD ---
        median_robust = np.median(band_data)
        std_robust = mad_std(band_data)
        stderr_robust = std_robust / np.sqrt(len(band_data))

        stats_data.append({
            'band': band,
            'n_band': len(band_data),
            'Mean (clip)': f"{mean_clip:.3f}",
            'Median (clip)': f"{median_clip:.3f}",
            'Std (clip)': f"{std_clip:.3f}",
            'StdErr (clip)': f"{stderr_clip:.3f}",
            'Median (robust)': f"{median_robust:.3f}",
            'Std (robust)': f"{std_robust:.3f}",
            'StdErr (robust)': f"{stderr_robust:.3f}"
        })
    else:
        stats_data.append({
            'band': band,
            'n_band': 0,
            'Mean (clip)': 'N/A',
            'Median (clip)': 'N/A',
            'Std (clip)': 'N/A',
            'StdErr (clip)': 'N/A',
            'Median (robust)': 'N/A',
            'Std (robust)': 'N/A',
            'StdErr (robust)': 'N/A'
        })

# Create DataFrame
stats_df = pd.DataFrame(stats_data)

# Display the full table
display(stats_df)
#print(stats_df.to_string(index=False))

# Display a truncated table
display(stats_df[['band', 'n_band', 'Mean (clip)', 'Median (clip)', 'StdErr (clip)']])


**Let's stop here for now:**

In [ ]:
print("Stopping here...")
raise StopExecution

## 9. Sandbox